In [1]:
import jarvis
import pandas as pd
import numpy as np

Waiting for redis server at 127.0.0.1:48327 to respond...
Waiting for redis server at 127.0.0.1:17854 to respond...
Starting local scheduler with 4 CPUs, 0 GPUs

View the web UI at http://localhost:8889/notebooks/ray_ui93737.ipynb?token=3ac759cc4278b08a2fdcb61fbaed4907804b1f55e4e6e430



# Where do I start? Summarize current and past experiments

In [2]:
summary = jarvis.versionSummaries('twitter_demo')

100%|██████████| 1/1 [00:00<00:00, 18.00it/s]


In [3]:
summary

,__trialNum__,alpha,frac,model,model_accuracy,split_seed,__commitHash__
0,0,0.000000,0.75,. . .,0.78794,42,90bee24da6bea15164e562c429fcfeadd5e08537
1,1,0.142857,0.75,. . .,0.75478,42,90bee24da6bea15164e562c429fcfeadd5e08537
2,2,0.285714,0.75,. . .,0.73327,42,90bee24da6bea15164e562c429fcfeadd5e08537
3,3,0.428571,0.75,. . .,0.71146,42,90bee24da6bea15164e562c429fcfeadd5e08537
4,4,0.571429,0.75,. . .,0.69731,42,90bee24da6bea15164e562c429fcfeadd5e08537
5,5,0.714286,0.75,. . .,0.68776,42,90bee24da6bea15164e562c429fcfeadd5e08537
6,6,0.857143,0.75,. . .,0.67680,42,90bee24da6bea15164e562c429fcfeadd5e08537
7,7,1.000000,0.75,. . .,0.66765,42,90bee24da6bea15164e562c429fcfeadd5e08537


# What is the best experiment?
Here, we measure _goodness_ of an experiment by its max model accuracy, over all trials.

In [4]:
summary_stat = summary.groupby('__commitHash__')['model_accuracy'].max()
summary_stat

__commitHash__
90bee24da6bea15164e562c429fcfeadd5e08537    0.78794
Name: model_accuracy, dtype: float64

In [5]:
best_commit_hash = summary_stat.idxmax()
best_commit_hash[0:5] + '...'

'90bee...'

# What is the best trial, given the best experiment?

In [6]:
best_index = summary.loc[summary['__commitHash__'] == best_commit_hash]['model_accuracy'].idxmax()
best_trial = summary.iloc[best_index]['__trialNum__']
best_trial

'0'

# So the former experiment is better than the latter. What changed?

In [7]:
versions = summary.__commitHash__.unique()
most_recent, least_recent = versions

In [8]:
jarvis.diffExperimentVersions('twitter_demo', least_recent, most_recent)

train_model.py --> train_model.py
@@ -47,8 +47,8 @@ def train(tweet_df, alpha):
     ## Convert tweet to bag of words for learning
 
     # Tokenize Text
-    vectorizer = TfidfVectorizer()
-    #vectorizer = CountVectorizer()
+    #vectorizer = TfidfVectorizer()
+    vectorizer = CountVectorizer()
 
     X_train = vectorizer.fit_transform(tweet_df["tweet"])
 


# Let's get the best model yet!
From the best trial, from the best experiment

In [9]:
best_model = jarvis.checkoutArtifact('twitter_demo', best_trial, best_commit_hash, 'model.pkl')

# Here's the best model in action

In [10]:
country_dict = best_model['country_dict']
classifier = best_model['classifier']
vectorizer = best_model['vectorizer']

code_dict = {}

for kee in country_dict:
    code_dict[country_dict[kee]] = kee

while True:
    tweet = input("What's on your mind? ")
    if tweet == 'exit':
        break
    tweet_vec = vectorizer.transform(np.array([tweet,]))
    country_id = classifier.predict(tweet_vec)
    print("Predicted country of origin: {}\n".format(code_dict[country_id[0]]))

What's on your mind? Hello world
Predicted country of origin:  US

What's on your mind? tqm
Predicted country of origin:  MX

What's on your mind? exit
